## Upload Trade History to Trade History Redis



This notebook uploads trade history data from BigQuery to our Memorystore trade history Redis instance. 

In [1]:
from google.cloud import bigquery
import os
import pandas as pd
import pickle
import redis
import tqdm
import numpy as np
#import dbtypes

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/creds.json"
bq_client = bigquery.Client()
project = "eng-reactor-287421"

In [3]:
def sqltodf(sql,limit = ""):
    if limit != "": 
        limit = f" ORDER BY RAND() LIMIT {limit}"
    bqr = bq_client.query(sql + limit).result()
    return bqr.to_dataframe()

The following query uses joins to retrive for any cusip only the record with the latest trade and the largest RTRS control number. Filtering on largest RTRS control number is necessary because trades sometimes occur simultaneously, and we want to select which of the simultaneous trades is selected in a predictable way. Later we may decide to select only those trades which are economically significant. The Cloud Function that updates our Redis database uses a simpler query with windowing to achieve a similar result becuase of the computing limitations BigQuery when it comes to self-joins in views.

In [20]:
%%time
df = sqltodf("select * from eng-reactor-287421.auxiliary_views.trade_history_groupby where recent[safe_offset(0)].publish_datetime < '2023-09-11' and recent[safe_offset(0)].calc_date is null"
            
            
            
            
            
            )

CPU times: user 11.7 s, sys: 2.36 s, total: 14.1 s
Wall time: 1min 21s


In [21]:
df.head()

,cusip,recent
0,70917RWR2,"[{'msrb_valid_from_date': 2020-07-23 09:22:10,..."
1,64970HBF0,"[{'msrb_valid_from_date': 2023-04-04 06:01:02,..."
2,13069FHT7,"[{'msrb_valid_from_date': 2019-06-03 12:53:37,..."
3,306126AE3,"[{'msrb_valid_from_date': 2021-01-06 13:22:04,..."
4,57559HAX5,"[{'msrb_valid_from_date': 2023-06-26 06:30:42,..."


In [4]:
trade_history_redis_host = os.environ.get('REDISHOST', '10.75.46.228')    # TODO: update this redis to have more memory and read replicas in similar vein to the current reference data redis (currently this redis is for testing)
trade_history_redis_port = int(os.environ.get('REDISPORT', 6379))
trade_history_redis_client = redis.Redis(host=trade_history_redis_host, port=trade_history_redis_port, db=0)

In [23]:
len(df)

206887

In [24]:
def upload_data_to_redis(key, value):
    value = pickle.dumps(value,protocol=pickle.HIGHEST_PROTOCOL)
    trade_history_redis_client.set(key, value)

In [25]:
trade_history_redis_client.dbsize()

1020302

In [5]:
trade_history_redis_client.exists("546594AG7")

1

In [27]:
FEATURES_FOR_EACH_TRADE_IN_HISTORY = ['msrb_valid_from_date', 
                                      'msrb_valid_to_date', 
                                      'rtrs_control_number', 
                                      'trade_datetime', 
                                      'publish_datetime', 
                                      'yield', 
                                      'dollar_price', 
                                      'par_traded', 
                                      'trade_type', 
                                      'is_non_transaction_based_compensation', 
                                      'is_lop_or_takedown', 
                                      'brokers_broker', 
                                      'is_alternative_trading_system', 
                                      'is_weighted_average_price', 
                                      'settlement_date', 
                                      'calc_date', 
                                      'calc_day_cat', 
                                      'maturity_date', 
                                      'next_call_date', 
                                      'par_call_date', 
                                      'refund_date', 
                                      'transaction_type',
                                      'sequence_number'
                                     ]


def convert_bq_trade_history_to_fast_redis_trade_history_format_and_upload_to_redis(df):
    for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        key = row['cusip']
        trade_history = row['recent']
        trade_history = np.array([[trade[feature] for feature in FEATURES_FOR_EACH_TRADE_IN_HISTORY] for trade in trade_history])
        try:
            upload_data_to_redis(key, trade_history)
        except Exception as e:
            raise e

In [28]:
convert_bq_trade_history_to_fast_redis_trade_history_format_and_upload_to_redis(df)

100%|██████████| 206887/206887 [2:17:20<00:00, 25.10it/s]  


In [9]:
#Retriving data for a single cusip
x = redis_client.mget(['732203BJ2','732203BJ2'])

In [32]:
upload_data_to_redis("hi", "hi back")

In [6]:
x = pickle.loads(trade_history_redis_client.get('546594AG7'))

In [7]:
import sys
sys.getsizeof(x)

296

In [8]:
len(x)

1

In [9]:
x

array([[datetime.datetime(2022, 5, 18, 6, 1, 3),
        datetime.datetime(2100, 1, 1, 0, 0), 2022051111861000,
        datetime.datetime(2022, 5, 11, 14, 43, 46),
        datetime.datetime(2022, 5, 18, 6, 1, 3), 1.369, 66.438,
        Decimal('66485000.000000000'), 'S', False, False, None, False,
        False, datetime.date(2022, 5, 13), datetime.date(2052, 5, 1), 2,
        datetime.date(2052, 5, 1), None, None, None, 'R', 640.0]],
      dtype=object)

In [49]:
import numpy as np

In [57]:
x["recent"][0]["yield"] = None

In [58]:
x["recent"][0]["yield"]

In [59]:
x["cusip"] = "QQQQQQQQQ"

In [11]:
cusips_to_delete = ['896560VM8',
 '84553RBL8',
 '13068KDX2',
 '05248NMG2',
 '13067SHM6',
 '60534RDX4',
 '956874DC2',
 '77355PJ20',
 '44237LWN2',
 '417846AA0',
 '41423BGM4',
 '04779DAJ1',
 '462467SY8',
 '57589TDK8',
 '97709P5D2',
 '41423CBF2',
 '739776FS6',
 '3434874C5',
 '82102REZ4',
 '739776FP2',
 '821686XP2',
 '60630ABP3',
 '54458AEB3',
 '6400826S7',
 '46246JLY9',
 '882724E94',
 '91473HBF5',
 '739776FR8',
 '58401AAV2',
 '62621EQA9',
 '46615YGG4',
 '262741EE7',
 '739776FN7',
 '13068CDQ5',
 '687714HF5',
 '562062DM8',
 '087470AX8',
 '16532LNP7',
 '68179RVK5',
 '13068BFU6',
 '739776FM9',
 '675498KU2',
 '662902EE1',
 '91514SEM4',
 '641847CT3',
 '6678257V9',
 '46615YFB6',
 '64469RG95',
 '64711NHH1',
 '46616TRB3',
 '27102TLZ7',
 '44239UKJ2',
 '283822ST2',
 '73357AER0',
 '91513UR87',
 '91412XQF8',
 '44239LSY1',
 '529063TZ2',
 '44239LQN7',
 '77160EGQ0',
 '012440PA5',
 '98647SDN8',
 '79626UAB7',
 '420064DQ7',
 '91513UY97',
 '41418YRD9',
 '692154HH3',
 '13079UEN8',
 '54459QD28',
 '09658BYB1',
 '739776FQ0',
 '04778ABM0',
 '081077J83',
 '33765VDB0',
 '610661FX3',
 '91514CAS0',
 '54468VCW1',
 '44243AHJ4',
 '88213M5X3',
 '71779XAQ0',
 '19624BAB7',
 '048573MJ5',
 '283668CE9',
 '649674GS1',
 '44243AJG8',
 '44237LUX2',
 '79741VDN6',
 '66332PBF0',
 '98647QNK7',
 '05248NMT4',
 '358730DV8',
 '44241BNG3',
 '74539XLV2',
 '739776FU1',
 '44239UKA1',
 '739776FT4',
 '74442PNP6',
 '98647QNA9',
 '68179RUF7',
 '9818482U7',
 '03444PAC6']







In [16]:
trade_history_redis_client.exists('03444PAC6')

0

In [17]:
for cusip in cusips_to_delete:
    try:
        trade_history_redis_client.delete(cusip)
        print(f"Deleted CUSIP: {cusip}")
    except Exception as e:
        print(f"Error deleting CUSIP {cusip}: {e}")
    

Deleted CUSIP: 896560VM8
Deleted CUSIP: 84553RBL8
Deleted CUSIP: 13068KDX2
Deleted CUSIP: 05248NMG2
Deleted CUSIP: 13067SHM6
Deleted CUSIP: 60534RDX4
Deleted CUSIP: 956874DC2
Deleted CUSIP: 77355PJ20
Deleted CUSIP: 44237LWN2
Deleted CUSIP: 417846AA0
Deleted CUSIP: 41423BGM4
Deleted CUSIP: 04779DAJ1
Deleted CUSIP: 462467SY8
Deleted CUSIP: 57589TDK8
Deleted CUSIP: 97709P5D2
Deleted CUSIP: 41423CBF2
Deleted CUSIP: 739776FS6
Deleted CUSIP: 3434874C5
Deleted CUSIP: 82102REZ4
Deleted CUSIP: 739776FP2
Deleted CUSIP: 821686XP2
Deleted CUSIP: 60630ABP3
Deleted CUSIP: 54458AEB3
Deleted CUSIP: 6400826S7
Deleted CUSIP: 46246JLY9
Deleted CUSIP: 882724E94
Deleted CUSIP: 91473HBF5
Deleted CUSIP: 739776FR8
Deleted CUSIP: 58401AAV2
Deleted CUSIP: 62621EQA9
Deleted CUSIP: 46615YGG4
Deleted CUSIP: 262741EE7
Deleted CUSIP: 739776FN7
Deleted CUSIP: 13068CDQ5
Deleted CUSIP: 687714HF5
Deleted CUSIP: 562062DM8
Deleted CUSIP: 087470AX8
Deleted CUSIP: 16532LNP7
Deleted CUSIP: 68179RVK5
Deleted CUSIP: 13068BFU6


In [1]:
with open('6-30-neg-yield.csv') as f:
    new = pd.read_csv(f)

In [ ]:
new

,cusip
0,468312EU3
1,16876QBG3
2,35981VAK6
3,235308QR7
4,706877AD9
...,...
1041,455719CP7
1042,76856PBF5
1043,424682DV1
1044,262588HC3


In [ ]:
redis_client.get("64972GBW3")

In [39]:
redis_client.delete("741530HY8")

1

In [15]:
for cusip in new["cusip"]:
    x = redis_client.delete(cusip)

In [19]:
x = pickle.loads(redis_client.get('64972GBW3'))

TypeError: a bytes-like object is required, not 'NoneType'

In [64]:
redis_client.exists("488386JA2")

1